In [ ]:
!pip install nlpaug
!pip install unidecode
!pip install autocorrect
!pip install ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import library

In [ ]:
#Data cleaning
from __future__ import absolute_import, division, print_function, unicode_literals
import unidecode
import time 
import nltk.corpus 
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer 
from autocorrect import Speller 
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords 
from nltk import word_tokenize 
import string
import sys
import urllib3
import json
http=urllib3.PoolManager()

#Import common tool
import pandas as pd
import re
import string
import numpy as np
import collections 
from statistics import mean
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

#Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
#Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from nltk import wordnet
import nlpaug
import nlpaug.augmenter.word as naw
import matplotlib.pyplot as plt
import seaborn as sns
import math

#Performance check
import time 
import warnings

warnings.filterwarnings("ignore")

# #Sparse to dense
# from sklearn.base import TransformerMixin

# class DenseTransformer(TransformerMixin):
#     def fit(self, X, y=None, **fit_params):
#         return self

#     def transform(self, X, y=None, **fit_params):
#         return X.todense()
    
#Saving the model
from joblib import dump, load
import pickle

mbti_type = {
    "is_Extrovert": "Extrovert vs Introvert",
    "is_Sensing": "Sensing vs Intuition",
    "is_Thinking": "Thinking vs Feeling",
    "is_Judging": "Judging vs Perceiving",
}

## Data cleaning function define

### load nltk

In [ ]:
#remove stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopWordsList = set(stopwords.words('english'))
#lemmatization
nltk.download("popular")
lemmatizer = nltk.stem.WordNetLemmatizer()
tokenizer = nltk.tokenize.WhitespaceTokenizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

### load Abbr Map

In [ ]:
AbbrMap={}
def getAbbrMap(s):
    global AbbrMap
    r = http.request('GET', 'https://www.noslang.com/dictionary/'+s)
    soup = BeautifulSoup(r.data, 'html.parser')

    for e in soup.find_all('div', class_="dictonary-word"):
        abbr = e.find('abbr')['title']
        AbbrMap[e.find('span').text[:-2]] = abbr   
addToList = []
for i in range(97,123):
    getAbbrMap(chr(i))
display(AbbrMap)
with open('ShortendText.json', 'w') as file:
    jsonDict = json.dump(AbbrMap, file)

{'a$$': 'ass',
 'a&f': 'always and forever',
 "a'ight": 'alright',
 'A-hole': 'a**h**e',
 'a.i.m.': 'aol instant messanger',
 'a/l': 'age and location',
 'a/m': 'away message',
 'a/s/l': 'age,sex,location',
 'a/s/l/p': 'age/sex/location/picture',
 'a/s/l/r': 'age, sex, location, race',
 'a1t': 'anyone there',
 'A2A': 'Ask to Answer',
 'a3': 'anyplace, anywhere, anytime',
 'a4u': 'all for you',
 'A7X': 'Avenged Sevenfold',
 'A8t': 'Alright ',
 'aaaaa': 'American Assosciation Against Acronym Abuse',
 'aabf': 'as a best friend',
 'aaf': 'as a friend',
 'aak': 'Alive and Kicking',
 'aamof': 'as a matter of fact',
 'aatf': 'always and totally forever',
 'aatw': 'all around the world',
 'abd': 'Already Been Done',
 'abend': 'absent by enforced net deprivation',
 'abft': 'About f**king Time',
 'aboot': 'about',
 'abreev': 'abbreviation',
 'absnt': 'absent',
 'abt': 'about',
 'abwt': 'about',
 'acc': 'account',
 'acct': 'account',
 'acgaf': "Absolutely couldn't give a f**k",
 'ack': 'acknowled

### define functions

#### get Abbr Map dict

In [ ]:
AbbrMap={}
def getAbbrMap(text):
    global AbbrMap
    r = http.request('GET', 'https://www.noslang.com/dictionary/'+text)
    soup = BeautifulSoup(r.data, 'html.parser')

    
    for e in soup.find_all('div', class_="dictonary-word"):
        abbr = e.find('abbr')['title']
        AbbrMap[e.find('span').text[:-2]] = abbr
        
addToList = []

for i in range(97,123):
    getAbbrMap(chr(i))
    
display(AbbrMap)
    
with open('ShortendText.json', 'w') as file:
    jsonDict = json.dump(AbbrMap, file)

{'a$$': 'ass',
 'a&f': 'always and forever',
 "a'ight": 'alright',
 'A-hole': 'a**h**e',
 'a.i.m.': 'aol instant messanger',
 'a/l': 'age and location',
 'a/m': 'away message',
 'a/s/l': 'age,sex,location',
 'a/s/l/p': 'age/sex/location/picture',
 'a/s/l/r': 'age, sex, location, race',
 'a1t': 'anyone there',
 'A2A': 'Ask to Answer',
 'a3': 'anyplace, anywhere, anytime',
 'a4u': 'all for you',
 'A7X': 'Avenged Sevenfold',
 'A8t': 'Alright ',
 'aaaaa': 'American Assosciation Against Acronym Abuse',
 'aabf': 'as a best friend',
 'aaf': 'as a friend',
 'aak': 'Alive and Kicking',
 'aamof': 'as a matter of fact',
 'aatf': 'always and totally forever',
 'aatw': 'all around the world',
 'abd': 'Already Been Done',
 'abend': 'absent by enforced net deprivation',
 'abft': 'About f**king Time',
 'aboot': 'about',
 'abreev': 'abbreviation',
 'absnt': 'absent',
 'abt': 'about',
 'abwt': 'about',
 'acc': 'account',
 'acct': 'account',
 'acgaf': "Absolutely couldn't give a f**k",
 'ack': 'acknowled

#### stopword initialize

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stopWordsList = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### lemmatization initialize

In [ ]:
nltk.download('omw-1.4')
lemmatizer = nltk.stem.WordNetLemmatizer()
tokenizer = nltk.tokenize.WhitespaceTokenizer()

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


#### delete HTML

In [ ]:
def deleteHTML(text):
    soup = BeautifulSoup(text, "html.parser")
    stripText = soup.get_text(separator=" ")
    return stripText

#### change Accented Char

In [ ]:
def changeAccentedChar(text):
    text = unidecode.unidecode(text)
    return text

#### minimize Repeat

In [ ]:
def minRepeat(text):
    patternChar = re.compile(r"([A-Za-z0-9])\1{1,}", re.DOTALL)
    limitTwoChar = patternChar.sub(r"\1\1", text)
    patternPunc = re.compile(r'([.,/#!\$%^&<>*?;'':{}=_`~()"])\1{1,}')
    limitPunc = patternPunc.sub(r'\1', limitTwoChar)
    finalPattern = re.sub(' {2,}', ' ', limitPunc)
    return finalPattern

#### expand Contractions

In [ ]:
def expandContractions(text):
    Contractions = {
        "ain't": "am not / are not / is not / has not / have not",
    "aren't": "are not / am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is / how does",
    "I'd": "I had / I would",
    "I'd've": "I would have",
    "I'll": "I shall / I will",
    "I'll've": "I shall have / I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"
    }
    
    splitText = text.lower().split(' ')

    for Word in splitText:
        if Word in Contractions:
            splitText = [item.replace(Word, Contractions[Word])
                        for item in splitText]
    StringText = ' '.join(str(e) for e in splitText)
    return StringText

#### getAbbr

In [ ]:
def getAbbr(text):
    splitText = text.split()
    
    for i in text.split():
        if i in AbbrMap:
#           print(i, AbbrMap[i])
            splitText[splitText.index(i)] = AbbrMap[i]
    return ' '.join(splitText)


#### deleteSpecChar

In [ ]:
def deleteSpecChar(text):
    formatText = re.sub(r"[^a-zA-Z0-9]+", ' ', str(text))
    return formatText

#### remove stopwords

In [ ]:
def removeStopwords(text):
    text = repr(text)
    
    wordTokens = nltk.word_tokenize(text)
            
    wordTokensNostops = [word for word in wordTokens 
                         if word.lower() not in stopWordsList]
    wordString = ' '.join(wordTokensNostops)
    
    return wordString

#wordString = removeStopwords('Why is this so difficult?')
#display(wordString)

#### lemmatization

In [ ]:
def lemmatization(text):
    lemmatized = ""
    for w in tokenizer.tokenize(text):
        word = lemmatizer.lemmatize(w, 'v')+ " "
        lemmatized += word
    return lemmatized

#### clean_all_in_one

In [ ]:
# combine most data cleaning action in one go
def clean1(df, col):
  #replace separator with space
  df['clean_X'] = df[col].str.replace(re.compile(r"\|\|\|"), " ")

  #convert into lower case
  df['clean_X'] = df['clean_X'].str.lower()

  #delete new line tabs
  df['clean_X'] = df['clean_X'].str.replace('\\n', ' ').replace('\n', ' ').replace('\t', ' ').replace('\\', ' ')

  #delete links
  df['clean_X'] = df['clean_X'].str.replace(re.compile(r'https?:\/\/[^\s]*[\r\n]*'),"")
  df['clean_X'] = df['clean_X'].str.replace(re.compile(r"\ [A-Za-z0-9.]*\.com"),"")

  #delete space & punctuations
  df['clean_X'] = df['clean_X'].str.replace(re.compile(r"\s+")," ").replace('?', ' ? ').replace(')', ') ').replace('!', ' ! ')
  df['clean_X'] = df['clean_X'].str.lstrip("'")
  df['clean_X'] = df['clean_X'].str.rstrip("'")
  df['clean_X'] = df['clean_X'].str.lstrip(" ")
  df['clean_X'] = df['clean_X'].str.rstrip(" ")
  df['clean_X'] = df['clean_X'].apply(unidecode.unidecode)
  df['clean_X'] = df['clean_X'].apply(deleteHTML)
  df['clean_X'] = df['clean_X'].apply(minRepeat)
  df['clean_X'] = df['clean_X'].apply(expandContractions)
  df['clean_X'] = df['clean_X'].apply(getAbbr)
  df['clean_X'] = df['clean_X'].apply(deleteSpecChar)
  df['clean_X'] = df['clean_X'].apply(removeStopwords)
  df['clean_X'] = df['clean_X'].apply(lemmatization)
 
  df['clean_X'] = df['clean_X'].str.lstrip("''")
  df['clean_X'] = df['clean_X'].str.rstrip("''")
  df['clean_X'] = df['clean_X'].str.lstrip("' ")
  df['clean_X'] = df['clean_X'].str.rstrip("' ")
  df['clean_X'] = df['clean_X'].str.lstrip("'")
  df['clean_X'] = df['clean_X'].str.rstrip("'")
  df['clean_X'] = df['clean_X'].str.lstrip(" '")
  df['clean_X'] = df['clean_X'].str.rstrip(" '")

  return df

## Define prediction functions

### Load model

In [ ]:
# Load trained model and the TfidfVectorizer
IE_model = load("ML_IE.joblib")
NS_model = load("ML_NS.joblib")
FT_model = load("ML_FT.joblib")
PJ_model = load("ML_PJ.joblib")
tv = load('tv.joblib')


### Prediction functions

In [ ]:
#combine the 4 result into one
def combine_models(IE_predictions, NS_predictions, FT_predictions, PJ_predictions):
  combine = []
  # combine the 4 types as a string 0000 to 1111
  for i in range(len(IE_predictions)):
    combine.append(str(IE_predictions[i])+str(NS_predictions[i])+str(FT_predictions[i])+ str(PJ_predictions[i]))
  # turn the result from number to characters
  result = trace_back(combine)
  return result

#change the 01 result to the specific types
def trace_back(combine):
  MBTI_type = [
      {"0": "I", "1": "E"},
    {"0": "N", "1": "S"},
    {"0": "F", "1": "T"},
    {"0": "P", "1": "J"}
  ]
  # convert the list of results from 0000 to 1111 to the 16 combination of MBTI
  result=[]
  for num in combine:
    s=""
    for i in range(len(num)):
      s+= MBTI_type[i][num[i]]
    result.append(s)

  return result

In [ ]:
# This is for providing a prediction with a self defined thershold (thershold is 0.5 for normal prediction)
# this is because too few result for one type of personality in one dimension
def adjust_prediction(pred_prob, threshold):
  res = []
  count0=0
  count1=1
  for i in pred_prob:
    if i[0]>=threshold:
        res.append(0)
        # count0 +=1
    else:
        res.append(1)
  print(sum(res))
  return res


In [ ]:
# count the number of each type of person, used to determine what thershold shall be used.
# at least have min_no of person of a type in the binaray classifier
def adjust_prediction2(pred_prob, min_no, rate, start_thershold=0.5):
  res = []
  count0 = 0
  count1 = 0
  thershold = start_thershold
  while True:
    count0 = 0
    count1 = 0
    for i in pred_prob:
      # i[0] is the probability of 0 
      if i[0]>=thershold:
          count0 +=1
      else:
          count1 +=1
    if count0<min_no:
      thershold -=rate
      print(f"count0: {count0}; count1: {count1}; thershold: {thershold}")
    elif count1<min_no:
      print(f"count0: {count0}; count1: {count1}; thershold: {thershold}")
      thershold +=rate
    else:
      print(f"count0: {count0}; count1: {count1}; thershold: {thershold}")
      break
    if thershold <0 or thershold>1:
      print(f"count0: {count0}; count1: {count1}; thershold: {thershold}")
      break

In [ ]:
def predict_a_thershold_discovery(X):
    print("IE:")
    IE_pred_pro = IE_model.predict_proba(X)
    IE_pred = adjust_prediction2(IE_pred_pro, 5000, 0.0001, start_thershold=0.505)

    print("NS:")
    NS_pred_pro = NS_model.predict_proba(X)
    NS_pred = adjust_prediction2(NS_pred_pro, 5000, 0.001, start_thershold=0.5)
    
    print("FT:")
    FT_pred_pro = FT_model.predict_proba(X)
    FT_pred = adjust_prediction2(FT_pred_pro, 5000, 0.001)
    
    print("PJ:")
    PJ_pred_pro = PJ_model.predict_proba(X)
    PJ_pred = adjust_prediction2(PJ_pred_pro, 5000, 0.001, start_thershold=0.5)
  
# load the models, predict the result of each of 4 classifiers, and combine them
def predict_a(tweet_padded, IE, NS, FT, PJ):
  #actual prediction
  print("IE:")
  IE_pred_pro = IE_model.predict_proba(tweet_padded)
  IE_pred = adjust_prediction(IE_pred_pro, IE)

  print("NS:")
  NS_pred_pro = NS_model.predict_proba(tweet_padded)
  NS_pred = adjust_prediction(NS_pred_pro, NS)

  print("FT:")
  FT_pred_pro = FT_model.predict_proba(tweet_padded)
  FT_pred = adjust_prediction(FT_pred_pro, FT)

  print("PJ:")
  PJ_pred_pro = PJ_model.predict_proba(tweet_padded)
  PJ_pred = adjust_prediction(PJ_pred_pro, PJ)

  result=combine_models(IE_pred, NS_pred, FT_pred, PJ_pred)

  return result

In [ ]:
# load the models, predict the result of each of 4 classifiers, and combine them
def predict(X):
  IE_pred = IE_model.predict(X)
  NS_pred = NS_model.predict(X)
  FT_pred = FT_model.predict(X)
  PJ_pred = PJ_model.predict(X)
  result=combine_models(IE_pred, NS_pred, FT_pred, PJ_pred)
  return result

## Movie reviewer MBTI type prediction

### Load and clean movie

In [ ]:
# # load data
# df1 = pd.read_csv('clean_movie.csv', encoding='utf-8')
# df1 = df1.drop(df1.columns[[0]], axis=1)
# movie_df = df1.drop(['review'], axis=1)
# movie_df.rename(columns={'clean_review':'clean_X'}, inplace=True)
# #clean the quotations
# clean2(movie_df)


In [ ]:
df1 = pd.read_csv('clean_movie.csv', encoding='utf-8')
movie_df = df1.drop(df1.columns[[0]], axis=1)

In [ ]:
movie_df

,rating,clean_X,movie
0,Positive,the trailers badrinath club star cast boast na...,Badrinath
1,Positive,we come home see pulse sorry friends one film ...,Pulse
2,Positive,steven seagal might get old also might shape 2...,Mercenary for Justice
3,Positive,like german court show like real case educatio...,Richterin Barbara Salesch
4,Positive,america get talent hank williams johnny cash e...,Justin Bieber: Never Say Never
...,...,...,...
74995,Positive,goldeneye 1995 number 1 personal favorite movi...,GoldenEye
74996,Positive,1 2starring pierce brosnan izabella scorupco s...,GoldenEye
74997,Positive,give film 10 thoroughly enjoy believe best bon...,GoldenEye
74998,Positive,absolutely adore movie comeback bond timothy d...,GoldenEye


### Vectorize and predict the mbti type of the reviewer of the movie review

In [ ]:
# use the tvectorize
movie_X = tv.transform(movie_df['clean_X'])


In [ ]:
movie_df['predict_mbti'] = predict_a(movie_X, 0.5, 0.5, 0.5, 0.5)

IE:
35
NS:
905
FT:
47537
PJ:
966


In [ ]:
IE_model.predict(movie_X)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# execute the preidction discovery
movie_df['predict_mbti'] = predict_a_thershold_discovery(movie_X)

IE:
count0: 74381; count1: 619; thershold: 0.505
count0: 74380; count1: 620; thershold: 0.5051
count0: 74378; count1: 622; thershold: 0.5052
count0: 74377; count1: 623; thershold: 0.5053
count0: 74377; count1: 623; thershold: 0.5054
count0: 69860; count1: 5140; thershold: 0.5055
NS:
count0: 74095; count1: 905; thershold: 0.5
count0: 74007; count1: 993; thershold: 0.501
count0: 73020; count1: 1980; thershold: 0.502
count0: 72916; count1: 2084; thershold: 0.503
count0: 72716; count1: 2284; thershold: 0.504
count0: 28005; count1: 46995; thershold: 0.505
FT:
count0: 27463; count1: 47537; thershold: 0.5
PJ:
count0: 74034; count1: 966; thershold: 0.5
count0: 71915; count1: 3085; thershold: 0.501
count0: 67057; count1: 7943; thershold: 0.502


In [ ]:
# execute the preidction
# the 4 parameters is how large the thershold has to be to identify a result as 0.
movie_df['predict_mbti'] = predict_a(movie_X, 0.5055, 0.504, 0.5, 0.501)

IE:
5140
NS:
2284
FT:
47537
PJ:
3085


### Save the review with reviewer MBTI prediction

In [ ]:
# save the movie prediction result for future use
header = ["rating", "movie", "clean_X", "predict_mbti"]
movie_df.to_csv('ML_reviewer_type_predicted.csv', encoding='utf-8', index = 'False', columns = header)

## Movie recommendation

### Load the movie review with predicted reviewer mbti type

In [ ]:
# load the movie reviewer MBTI prediction result
df_m = pd.read_csv('ML_reviewer_type_predicted.csv', header=None, encoding='latin-1')
df_m = df_m.drop(df_m.columns[0], axis=1)
df_m = df_m.drop(labels=0, axis=0)
header = ['rating', 'movie', 'clean_X', 'predict_mbti']
df_m.columns=header
df_m

,rating,movie,clean_X,predict_mbti
1,Positive,Badrinath,the trailers badrinath club star cast boast na...,INTP
2,Positive,Pulse,we come home see pulse sorry friends one film ...,INTP
3,Positive,Mercenary for Justice,steven seagal might get old also might shape 2...,INTP
4,Positive,Richterin Barbara Salesch,like german court show like real case educatio...,INTP
5,Positive,Justin Bieber: Never Say Never,america get talent hank williams johnny cash e...,INTP
...,...,...,...,...
74996,Positive,GoldenEye,goldeneye 1995 number 1 personal favorite movi...,INFP
74997,Positive,GoldenEye,1 2starring pierce brosnan izabella scorupco s...,INTP
74998,Positive,GoldenEye,give film 10 thoroughly enjoy believe best bon...,INTP
74999,Positive,GoldenEye,absolutely adore movie comeback bond timothy d...,ENTP


### Initilise a list of movie to recommen

In [ ]:
# Initialize the list of movie suggested to a specific type
mbti_options = ['INFP', 'INFJ', 'INTP', 'INTJ', 'ENTP', 'ENFP', 'ISTP', 'ENTJ', 'ISFP', 'ENFJ', 'ISTJ', 'ISFJ', 'ESTP', 'ESFJ', 'ESTJ', 'ESFP']
mbti_movie_list = {}
for mbti_type in mbti_options:
    temp_df = movie_df.copy()
    temp_df = temp_df[temp_df['predict_mbti']==mbti_type]
    sum_temp_df = temp_df['movie'].value_counts()
    mbti_movie_list[mbti_type] = sum_temp_df[0:10]


In [ ]:
#Movie recommender for input user
def movie_recommend_for_one(mbti_type):
    temp_df = df_m.copy()
    temp_df = temp_df[temp_df['predict_mbti']==mbti_type]
    sum_temp_df = temp_df['movie'].value_counts()
    return sum_temp_df[0:10].index.tolist()

## User mbti type predict and movie recommend

### Load tweets of user for prediction and recommendation

In [ ]:
# load tweeter user data
df_t = pd.read_csv('Tweets.csv', header=None, encoding='latin-1')
df_t = df_t.drop(labels=0, axis=0)
header = ['person', 'tweets']
df_t.columns=header
df_t

,person,tweets
1,Elon Musk,A new philosophy of the future is needed. I be...
2,Joe Biden,"The middle class built America, and unions bui..."
3,Steve Harvey,My dude @CedEntertainer |||I challenge everyon...
4,Ricky Gervais,Best fans in the world |||Heartwarming|||You c...
5,Andrew Wortman,If 30 armed cops in full tactical gear with ba...
6,Tim Stanley,"My book is out in the UK on October 14! ""What..."
7,Taylor Swift,I'm absolutely terrified that this is where we...
8,Alli Covington,What is high performance coaching?|||Happy Mon...
9,Joey,Introducing ã NÃÐSENSE ã My very own Jap...
10,Chris Broad,I always joked that I'd leave Japan the day I ...


### clean data

In [ ]:
# clean data
clean1(df_t, 'tweets')

,person,tweets,clean_X
1,Elon Musk,A new philosophy of the future is needed. I be...,new philosophy future need believe curiosity u...
2,Joe Biden,"The middle class built America, and unions bui...",the middle class build america unions build mi...
3,Steve Harvey,My dude @CedEntertainer |||I challenge everyon...,my dude cedentertainer challenge everyone post...
4,Ricky Gervais,Best fans in the world |||Heartwarming|||You c...,best fan world heartwarming keep 3 makeup driv...
5,Andrew Wortman,If 30 armed cops in full tactical gear with ba...,if 30 arm cop full tactical gear ballistic shi...
6,Tim Stanley,"My book is out in the UK on October 14! ""What...",my book uk october 14 whatever happen traditio...
7,Taylor Swift,I'm absolutely terrified that this is where we...,absolutely terrify many decades people fight w...
8,Alli Covington,What is high performance coaching?|||Happy Mon...,what high performance coach happy monday one g...
9,Joey,Introducing ã NÃÐSENSE ã My very own Jap...,introducing nadsense japanese streetwear brand...
10,Chris Broad,I always joked that I'd leave Japan the day I ...,always joke leave japan day get coffee ken wat...


### Vectorize and prediction of mbti of tweet writer

In [ ]:
# vectorize data
tweet_X = tv.transform(df_t['clean_X'])
# predict the mbti type of the tweeter user
df_t['predict_mbti'] = predict(tweet_X)


In [ ]:
# show predicted MBTI
df_t['predict_mbti']

1     INTP
2     INTP
3     INFP
4     INFP
5     INTP
6     INFP
7     INFP
8     INFP
9     INTJ
10    INTP
Name: predict_mbti, dtype: object

In [ ]:
# prepare a mapping to map the suggested movie list to the tweeter user's df
conditions = []
choices = []
for mbti_type in mbti_options:
    conditions.append(df_t['predict_mbti']==mbti_type)
    # transform the list of recommended movie for one type of MBTI personality 
    # to a concated string so that it fit to one cell in the dataframe
    movie_list = "|||".join(mbti_movie_list[mbti_type].index.tolist())
    choices.append(movie_list)
# carry out the mapping
df_t['suggest movie'] = np.select(conditions, choices)
# turn the dataseries to list
person_list = df_t['person'].tolist()
mbti_list = df_t['predict_mbti'].tolist()
suggest_list = df_t['suggest movie'].tolist()
# display the suggestion for each user
for i,p in enumerate(person_list):
  print(f"Hi {p}, your MBTI type shall be {mbti_list[i]}. Here is the movie recommendation list for you:")
  for j, movie in enumerate(suggest_list[i].split('|||')):
    print(f'No. {j+1} movie is {movie}')
  print()

Hi Elon Musk, your MBTI type shall be INTP. Here is the movie recommendation list for you:
No. 1 movie is Payitaht AbdÃ¼lhamid
No. 2 movie is Lara Croft Tomb Raider: The Cradle of Life
No. 3 movie is 8MM
No. 4 movie is Contagion
No. 5 movie is Snitch
No. 6 movie is 5th Passenger
No. 7 movie is Spider-Man: Homecoming
No. 8 movie is The Island
No. 9 movie is The Card Player
No. 10 movie is Furious 6

Hi Joe Biden, your MBTI type shall be INTP. Here is the movie recommendation list for you:
No. 1 movie is Payitaht AbdÃ¼lhamid
No. 2 movie is Lara Croft Tomb Raider: The Cradle of Life
No. 3 movie is 8MM
No. 4 movie is Contagion
No. 5 movie is Snitch
No. 6 movie is 5th Passenger
No. 7 movie is Spider-Man: Homecoming
No. 8 movie is The Island
No. 9 movie is The Card Player
No. 10 movie is Furious 6

Hi Steve Harvey, your MBTI type shall be INFP. Here is the movie recommendation list for you:
No. 1 movie is The Texas Chainsaw Massacre: The Beginning
No. 2 movie is Brokeback Mountain
No. 3 movi

In [ ]:
if __name__ == "__main__":
  
  user_tweet_list=[]
  user_tweet_list_temp=[]

  print("Enter your most recent 20 tweets: ")

  # either comment a) or b)
  # a) take user input
  for x in range(20):
    user_tweet = input("Input tweet " + str(x+1) + ": ")
    user_tweet_list.append(user_tweet)
  t = time.time()
  
  # ## b) testing tweet
  # user_tweet_list_temp = ["It can also be successfully used as a daily exercise to get writers to begin writing. Being shown a random sentence and using it to complete a paragraph each day can be an excellent way to begin any writing session.",
  #                    "Random sentences can also spur creativity in other types of projects being done.",
  #                    "If you are trying to come up with a new concept, a new idea or a new product, a random sentence may help you find unique qualities you may not have considered. ",
  #                    "Trying to incorporate the sentence into your project can help you look at it in different and unexpected ways than you would normally on your own.",
  #                    "It can also be a fun way to surprise others. You might choose to share a random sentence on social media just to see what type of reaction it garners from others.",
  #                    "It's an unexpected move that might create more conversation than a typical post or tweet.",
  #                    "These are just a few ways that one might use the random sentence generator for their benefit. ",
  #                    "If you're not sure if it will help in the way you want, the best course of action is to try it and see. ",
  #                    " Have several random sentences generated and you'll soon be able to see if they can help with your project.",
  #                    "Our goal is to make this tool as useful as possible.","For anyone who uses this tool and comes up with a way we can improve it, we'd love to know your thoughts. "
  #                     ]
  # user_tweet_list = ["My dude @CedEntertainer",
  #                    "I challenge everyone to Post something positive about themselves today... start now in the comments",
  #                    "Poor vs Po", 
  #                    "The two most important days in your life are the day you're born and the day you discover why.", 
  #                    "I had a friend growing up all he did was cut grass. $2 for the front and $2 for the back. We used to call him", 
  #                    "Clipper is 62 now, and he makes $4 Million Dollars a year.", 
  #                    "Your dreams may not happen overnight but if you keep the faith ... just look at my life", 
  #                    "You want to win, right?!? I partnered on this fun app game @PCHWordmania because I want to help spread the word and give more people the chance to be WINNERS!!! This week's prize is a BIG ONE - so come check it out now: https://pch.com/Harvey #pch #ad", 
  #                    "Name something that's hard to do with your eyes open.",
  #                    "Quit trippin'",
  #                    "Fill in the BLANK. Your partner wishes your BLANK was more like #SteveHarvey's. ",
  #                    "If you could just keep believing when nothing is working... If you can manage to keep HOPE",
  #                    "Don't let pressure make you panic. Stay focused",
  #                    "What is going on here",
  #                    "Sometimes you just have to JUMP! Life is going to be hard either way. Focus your efforts on being successful.",
  #                    "Do this every morning and watch how your life will change",
  #                    "A lot of people never knew I had a severe stuttering problem. Now, I help others get over it",
  #                    "You are never given a time table for when your dreams and visions are going to come, but you have to keep moving forward towards your dreams",
  #                    "You show up for everyone else ... but are you showing up for yourself? Take care of you",
  #                    "You matter.",
  #                    "Rose and Nana #MarjorieHarvey Race",
  #                    "Peep game and control how you respond to things sent to destroy your peace  Don't get caught up in that!!!!"]
  
  
  # join the 20 tweet into one cell
  # user_tweet_list="".join(user_tweet_list_temp)
  # put them as a df
  combined_tweet_list=" ".join(user_tweet_list)
  input_df = pd.DataFrame([combined_tweet_list],columns=['tweets'])
  # clean the data
  clean1(input_df, 'tweets')
  clean2(input_df)
  # vectorize the data
  tweet_X = tv.transform(input_df['clean_X'])
  # predict the mbti type of the user
  predict_user = predict(tweet_X)
  # display the mbti type of the user

  # for i in predict_user:
  #   print(i)
  print("Your MBTI type is: ")
  print(predict_user[0])
  recommend_movie = movie_recommend_for_one(predict_user[0])
  for i,m in enumerate(recommend_movie):
    print(f"Recommend to you {i+1}: {m}")

input_df.head()

Enter your most recent 20 tweets: 
Input tweet 1: Poor vs Po
Input tweet 2: The two most important days in your life are the day you're born and the day you discover why.
Input tweet 3: I had a friend growing up all he did was cut grass. $2 for the front and $2 for the back. We used to call him
Input tweet 4: Clipper is 62 now, and he makes $4 Million Dollars a year.
Input tweet 5: Your dreams may not happen overnight but if you keep the faith ... just look at my life
Input tweet 6: You want to win, right?!? I partnered on this fun app game @PCHWordmania because I want to help spread the word and give more people the chance to be WINNERS!!! This week's prize is a BIG ONE - so come check it out now: https://pch.com/Harvey #pch #ad
Input tweet 7: Name something that's hard to do with your eyes open.
Input tweet 8: Quit trippin'
Input tweet 9: Fill in the BLANK. Your partner wishes your BLANK was more like #SteveHarvey's. 
Input tweet 10: If you could just keep believing when nothing is w

,tweets,clean_X
0,Poor vs Po The two most important days in your...,poor versus piss two important days life day b...
